![i2b2 Logo](images/transmart-logo.png)


# Using LLM for Clinical Note Analysis (Zero-shot)
This Notebook demonstrates how to use Azure-hosted OpenAI models with LangChain. The notebook follows a step-by-step approach to set up authentication, initialize the model, create structured prompts, and generate responses. The key components covered include:#

- ## REVIEW !!! 
- Loading environment variables
- Setting up Azure authentication
- Creating and formatting chat prompts
- Invoking the OpenAI model
- Displaying the AI-generated response

Each cell in the notebook builds upon the previous one to progressively set up and execute an AI-driven conversation.

In [21]:
# -----------------------------------------------------------
# 1. Ollama Installation & Model Download
# -----------------------------------------------------------
# 📌 One-time setup instructions for workshop participants

# ⚙️ Step 1: Install Ollama
#   - Download from: https://ollama.com/download
#   - Follow your OS-specific install steps
#   - Restart your terminal after installation
#   - Verify install: run `ollama version` in terminal

# ⚙️ Step 2: Automatically pull the required model
#   - You can change the model name below if needed (e.g., "qwen3")

import subprocess

# Choose your model
model_name = "qwen2:7b"  # Change to "qwen3" if preferred

# Pull the model (safe to run multiple times)
print(f"📥 Downloading model '{model_name}' via Ollama...")
subprocess.run(["ollama", "pull", model_name], check=True)


📥 Downloading model 'qwen2:7b' via Ollama...


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 43f7a214e532: 100% ▕██████████████████▏ 4.4 GB                         
pulling 77c91b422cc9: 100% ▕██████████████████▏ 1.4 KB                         
pulling c156170b718e: 100% ▕██████████████████▏  11 KB                         
pulling f02dd72bb242: 100% ▕██████████████████▏   59 B                         
pulling 75357d685f23: 100% ▕██████████████████▏   28 B                         
pulling 648f809ced2b: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'qwen2:7b'], returncode=0)

In [22]:
# -----------------------------------------------------------
# 2. Initialize the Qwen Model via Ollama
# -----------------------------------------------------------
# Sets up the model interface using LangChain's Ollama wrapper

from langchain_community.chat_models import ChatOllama

# Use the same model name as in the previous cell
model = ChatOllama(model=model_name)

print(f"\n✅ Ollama Model '{model_name}' is loaded and ready to use.")



✅ Ollama Model 'qwen2:7b' is loaded and ready to use.


## 3. Basic Prompt Interaction
<img src="https://github.com/v-mourajr/mongan_llm_workshop/blob/master/images/basic_prompt.png?raw=true" alt="Basic Prompt" width="800">

In [23]:
# -----------------------------------------------------------
# 3. Basic Prompt Interaction
# -----------------------------------------------------------
# This cell shows how to send structured messages (system + user prompt)
# to the Qwen model using LangChain's Ollama integration.

from langchain_core.messages import HumanMessage, SystemMessage

# Define the system and user messages
messages = [
    SystemMessage(content=(
        "You are a knowledgeable medical provider with expertise in diagnosing and managing various diseases. "
        "Provide clear, evidence-based, and patient-friendly explanations about medical conditions, symptoms, and treatments."
    )),
    HumanMessage(content="What is asthma? What are its common symptoms and treatments?")
]

# Send the messages to the model and get the response
response = model.invoke(messages)

# Display the model's reply
print("🧠 Model Response:\n")
print(response.content)


🧠 Model Response:

**Asthma**

**Definition**: Asthma is a chronic inflammatory condition of the airways in your lungs. It causes recurring episodes of breathlessness, wheezing, chest tightness, or coughing. The inflammation leads to narrowing of the bronchial tubes (the airways), making it difficult for air to flow in and out of the lungs.

**Common Symptoms**: Asthma symptoms can vary widely between individuals and even from one episode to another within a person. Common symptoms include:

1. **Wheezing**: A whistling sound when breathing, heard most often during exhalation.
2. **Coughing**
3. **Shortness of breath (dyspnea)**
4. **Chronic or episodic chest tightness**
5. **Recurring nocturnal symptoms** (waking up due to difficulty in breathing)

These symptoms are typically triggered by irritants such as allergens, air pollution, exercise, cold temperatures, viral infections, and certain medications.

**Diagnosis**: Asthma is usually diagnosed based on the history of symptoms, phys

## 4. Using ChatPromptTemplate for Dynamic Queries
<img src="https://github.com/v-mourajr/mongan_llm_workshop/blob/master/images/prompt_template.png?raw=true" alt="Basic Prompt" width="800">


In [ ]:
# -----------------------------------------------------------
# 4. Using ChatPromptTemplate for Dynamic Queries
# -----------------------------------------------------------
# Demonstrates how to create a reusable prompt template with dynamic input variables using LangChain.

# Key Components:
#   - ChatPromptTemplate: Allows dynamic variables in prompts, making interactions more flexible.
#   - SystemMessage: Sets the AI’s role as a medical provider, ensuring responses are clear and evidence-based.
#   - HumanMessage: Contains a query template with a placeholder ({disease}), allowing different medical conditions to be queried dynamically.
#   - prompt_template.invoke({"disease": disease}): Fills in the variable "disease" with the specified condition (e.g., "epilepsy").
#   - model.invoke(prompt): Sends the formatted query to the AI model for a response.

# Purpose:
# This approach enables reusable prompts where users can query different diseases without modifying the prompt structure.
# It improves scalability and efficiency for medical applications or chatbots handling multiple medical conditions.


from langchain.prompts import ChatPromptTemplate

ai_role = "5 year-old"
disease = "epilepsy"

# messages = [
#     ("system", f"You are a knowledgeable {ai_role}. Provide clear, evidence-based, and patient-friendly explanations about medical conditions, symptoms, and treatments."),
# 
#     ("human", "What is {disease}? What are its common symptoms and treatments?"),
# ]

messages = [
    ("system", f"You are a knowledgeable medical provider. Provide clear and patient-friendly explanations about medical conditions, symptoms, and treatments, ensuring they are suitable for a {ai_role}."),

    ("human", "What is {disease}? What are its common symptoms and treatments?"),
]

# Create PromptTemplate
prompt_template = ChatPromptTemplate.from_messages(messages)

# Plug-in user variables
prompt = prompt_template.invoke({"ai_role": ai_role , "disease": disease})

# invoke model
result = model.invoke(prompt)
print(result.content)

In [ ]:
# -----------------------------------------------------------
# 5. Using ChatPromptTemplate for Clinical Note Analysis
# -----------------------------------------------------------
# Demonstrates how to create a structured prompt template for extracting medical details from clinical notes.

# Key Components:
#   - ChatPromptTemplate: Creates a reusable prompt structure for analyzing clinical notes.
#   - SystemMessage: Defines the AI’s role as a medical documentation assistant, ensuring accurate extraction of medical details.
#   - HumanMessage: Contains a structured query with a placeholder ({patient_note}), allowing different clinical notes to be analyzed dynamically.
#   - prompt_template_notes.invoke({"patient_note": note_text}): Fills in the variable "patient_note" with the actual clinical note text.
#   - model.invoke(prompt): Sends the formatted query to the AI model for processing.

# Purpose:
# This approach enables automated medical text processing, ensuring structured extraction of relevant information such as demographics, chief complaints, medications, and asthma status.
# It improves efficiency in clinical documentation and can be integrated into medical record systems for automated analysis.


messages_notes = [
    (
        "system", 
        "You are an advanced medical documentation assistant with expertise in clinical text analysis. Your task is to review a given clinical note and extract relevant medical details accurately."
    ),
    
    (
        "human", 
        "Please analyze the following clinical note: {patient_note}. \n\n"
        "Extract and list the following information:\n"
        "1. Patient demographics\n"
        "2. Chief Complaints\n"
        "3. Current Medications\n"
        "4. Determine whether the patient has asthma (Yes/No), based on explicit mentions or related diagnoses.\n\n"
        "Provide the output in a structured format."
    )
]

prompt_template_notes = ChatPromptTemplate.from_messages(messages_notes)


In [ ]:
# -----------------------------------------------------------
# 6. Loading and Reading a Patient Note
# -----------------------------------------------------------
# Demonstrates how to load a clinical note from a file for further processing.

# Key Components:
#   - input_dir: Specifies the directory where patient notes are stored.
#   - filename: Defines the specific file to be loaded, containing a patient's medical note.
#   - os.path.join(input_dir, filename): Constructs the full file path dynamically.
#   - open(file_path, "r"): Opens the file in read mode and loads the content into the variable "patient_note".
#   - print(f"\nPatient Note:\n\n{patient_note}"): Displays the loaded note for verification.

# Purpose:
# This step ensures that patient notes are correctly loaded before being analyzed by the AI model.
# It enables seamless integration with document processing pipelines for clinical text analysis.


input_dir = 'data_prep/patient_notes_old'
filename = 'note_1000000064_20090608.txt' 

# examples without Asthma: 
# -----------------------------------------------------------
# 1000000002, 1000000003, 1000000009, 1000000010, 1000000013, 
# 1000000023, 1000000036, 1000000040, 1000000047, 1000000048, 
# 1000000052, 1000000063, 1000000064, 1000000068, 1000000071, 
# 1000000082, 1000000086, 1000000087, 1000000093, 1000000101, 
# 1000000103, 1000000107
# -----------------------------------------------------------


# Load the note txt
file_path = os.path.join(input_dir, filename)
with open(file_path, "r") as file:
    patient_note = file.read()
    
print(f"\nPatient Note:\n\n{patient_note}")

In [ ]:
# -----------------------------------------------------------
# 7. Invoking the AI Model for Clinical Note Analysis
# -----------------------------------------------------------
# Demonstrates how to process a patient note using a structured prompt and invoke the AI model for analysis.

# Key Components:
#   - prompt_template_notes.invoke({"patient_note": patient_note}): 
#     Dynamically fills the prompt template with the actual clinical note text.
#   - model.invoke(prompt_notes): 
#     Sends the formatted query to the AI model for processing.
#   - print(result_note.content): 
#     Displays the AI-generated structured output containing extracted medical details.

# Purpose:
# This approach enables automated extraction of structured medical information from clinical notes.
# It ensures efficient and standardized processing of patient datafiles, aiding in medical documentation and decision-making.

prompt_notes = prompt_template_notes.invoke({"patient_note": patient_note})

# invoke model
result_note = model.invoke(prompt_notes)
print(result_note.content)
